<a href="https://colab.research.google.com/github/cruz-marco/pyspark_course/blob/main/pyspark_DIY01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Config do Spark

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.3/spark-3.2.3-bin-hadoop3.2.tgz
!tar xf spark-3.2.3-bin-hadoop3.2.tgz

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64" 
os.environ["SPARK_HOME"] = '/content/spark-3.2.3-bin-hadoop3.2'

!pip install -q findspark

import findspark
findspark.init()
findspark.find()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

# Importe do módulo de funções.

In [19]:
from pyspark.sql import functions as f

# Criação e verificação dos DataFrames

In [79]:
clientes = spark.read.load(('/content/drive/MyDrive/Datasets/pyspark_course/Ati'
                            'vidades/Clientes.parquet'), format="parquet")

itens_vendas = spark.read.load(("/content/drive/MyDrive/Datasets/pyspark_course"
                                "/Atividades/ItensVendas.parquet"), 
                                format="parquet")

produtos = spark.read.load(("/content/drive/MyDrive/Datasets/pyspark_course/Ati"
                            "vidades/Produtos.parquet"), format="parquet")

vendas = spark.read.load(("/content/drive/MyDrive/Datasets/pyspark_course/Ativ"
                          "idades/Vendas.parquet"), format="parquet")

vendedores = spark.read.load(("/content/drive/MyDrive/Datasets/pyspark_course/A"
                            "tividades/Vendedores.parquet"), format="parquet")

dataframes = [clientes, itens_vendas, produtos, vendas, vendedores]

In [80]:
for frame in dataframes:
  frame.show(5)

+---------+--------------------+------+------+--------+
|ClienteID|             Cliente|Estado|Genero|  Status|
+---------+--------------------+------+------+--------+
|        1|Adelina Buenaventura|    RJ|     M|  Silver|
|        2|        Adelino Gago|    RJ|     M|  Silver|
|        3|     Adolfo Patrício|    PE|     M|  Silver|
|        4|    Adriana Guedelha|    RO|     F|Platinum|
|        5|       Adélio Lisboa|    SE|     M|  Silver|
+---------+--------------------+------+------+--------+
only showing top 5 rows

+---------+--------+----------+-------------+----------+--------+----------------+
|ProdutoID|VendasID|Quantidade|ValorUnitario|ValorTotal|Desconto|TotalComDesconto|
+---------+--------+----------+-------------+----------+--------+----------------+
|        2|     400|         2|       9201.0|   18402.0| 6256,68|        12145.32|
|        2|     385|         2|       9201.0|   18402.0| 5704,62|        12697.38|
|        4|     395|         2|       6892.2|   13784.4|

# 1. Criar uma consulta que mostre nome, estado e status dos clientes.

In [81]:
clientes.select(f.col('Cliente').alias('Nome'), f.col('Estado'), f.col('Status')
                ).show()

+--------------------+------+--------+
|                Nome|Estado|  Status|
+--------------------+------+--------+
|Adelina Buenaventura|    RJ|  Silver|
|        Adelino Gago|    RJ|  Silver|
|     Adolfo Patrício|    PE|  Silver|
|    Adriana Guedelha|    RO|Platinum|
|       Adélio Lisboa|    SE|  Silver|
|       Adérito Bahía|    MA|  Silver|
|       Aida Dorneles|    RN|  Silver|
|   Alarico Quinterno|    AC|  Silver|
|    Alberto Cezimbra|    AM|  Silver|
|    Alberto Monsanto|    RN|    Gold|
|       Albino Canela|    AC|  Silver|
|     Alceste Varanda|    RR|  Silver|
|  Alcides Carvalhais|    RO|  Silver|
|        Aldo Martins|    GO|  Silver|
|   Alexandra Tabares|    MG|  Silver|
|      Alfredo Cotrim|    SC|  Silver|
|     Almeno Figueira|    SC|  Silver|
|      Alvito Peralta|    AM|  Silver|
|     Amadeu Martinho|    RN|  Silver|
|      Amélia Estévez|    PE|  Silver|
+--------------------+------+--------+
only showing top 20 rows



# 2. Crie uma consulta que mostre apenas os clientes do status Gold e Platinum

In [82]:
conditions = [(f.col('Status') == 'Gold'), 
              (f.col('Status') == 'Platinum')]

clientes.where(conditions[0] | conditions[1])\
        .orderBy(f.col('Status').asc())\
        .show()

+---------+-------------------+------+------+--------+
|ClienteID|            Cliente|Estado|Genero|  Status|
+---------+-------------------+------+------+--------+
|       68|      Carminda Dias|    AM|     F|    Gold|
|       10|   Alberto Monsanto|    RN|     M|    Gold|
|      166|   Firmino Meireles|    AM|     M|    Gold|
|      220|Honorina Villaverde|    PE|     F|    Gold|
|      237|  Iracema Rodríguez|    BA|     F|    Gold|
|       83|      Cláudio Jorge|    TO|     M|    Gold|
|       28|      Anna Carvajal|    RS|     F|    Gold|
|      121|    Dionísio Saltão|    PR|     M|    Gold|
|       49|      Bento Quintão|    SP|     M|    Gold|
|        4|   Adriana Guedelha|    RO|     F|Platinum|
|      170|      Flor Vilanova|    CE|     M|Platinum|
|      230|    Ibijara Botelho|    RR|     F|Platinum|
|      247|         Joana Ataí|    GO|     F|Platinum|
+---------+-------------------+------+------+--------+



# 3. Demonstre quanto cada Status de clientes representa em vendas.

Documentação do [Join](https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.sql.DataFrame.join.html)

In [83]:
vendas.join(clientes, vendas.ClienteID == clientes.ClienteID, how='inner')\
      .select(vendas.ClienteID, f.col('VendasID'), f.col('Total'), 
              f.col('Status'))\
      .groupBy('Status')\
      .agg(f.sum('Total').alias('Total/Status'), 
           f.mean('Total').alias('MedTotal/Status'), 
           f.count('VendasID').alias('Vendas/Status'))\
      .orderBy(f.col('Total/Status').desc())\
      .show()

+--------+------------------+-----------------+-------------+
|  Status|      Total/Status|  MedTotal/Status|Vendas/Status|
+--------+------------------+-----------------+-------------+
|  Silver|        3014291.36|7631.117367088607|          395|
|    Gold|27286.690000000002|9095.563333333334|            3|
|Platinum|          12584.68|          6292.34|            2|
+--------+------------------+-----------------+-------------+



In [84]:
vendas.join(clientes, vendas.ClienteID == clientes.ClienteID, how='inner')\
      .select(vendas.ClienteID, f.col('VendasID'), f.col('Total'), 
              f.col('Status'))\
      .groupBy('Status')\
      .agg(f.sum('Total').alias('Total/Status'), 
           f.mean('Total').alias('MedTotal/Status'), 
           f.count('VendasID').alias('Vendas/Status'))\
      .orderBy(f.col('MedTotal/Status').desc())\
      .show()

+--------+------------------+-----------------+-------------+
|  Status|      Total/Status|  MedTotal/Status|Vendas/Status|
+--------+------------------+-----------------+-------------+
|    Gold|27286.690000000002|9095.563333333334|            3|
|  Silver|        3014291.36|7631.117367088607|          395|
|Platinum|          12584.68|          6292.34|            2|
+--------+------------------+-----------------+-------------+



In [85]:
vendas.join(clientes, vendas.ClienteID == clientes.ClienteID, how='inner')\
      .select(vendas.ClienteID, f.col('VendasID'), f.col('Total'), 
              f.col('Status'))\
      .groupBy('Status')\
      .agg(f.sum('Total').alias('Total/Status'), 
           f.mean('Total').alias('MedTotal/Status'), 
           f.count('VendasID').alias('Vendas/Status'))\
      .orderBy(f.col('Vendas/Status').desc())\
      .show()

+--------+------------------+-----------------+-------------+
|  Status|      Total/Status|  MedTotal/Status|Vendas/Status|
+--------+------------------+-----------------+-------------+
|  Silver|        3014291.36|7631.117367088607|          395|
|    Gold|27286.690000000002|9095.563333333334|            3|
|Platinum|          12584.68|          6292.34|            2|
+--------+------------------+-----------------+-------------+



In [86]:
vendas.join(clientes, vendas.ClienteID == clientes.ClienteID, how='inner')\
      .select(vendas.ClienteID, f.col('VendasID'), f.col('Total'), 
              f.col('Status'))\
      .groupBy('Status')\
      .agg(f.sum('Total').alias('Total/Status'), 
           f.mean('Total').alias('MedTotal/Status'), 
           f.count('VendasID').alias('Vendas/Status'))\
      .withColumn('% Volume', (f.col('Vendas/Status') / vendas.count()) * 100)\
      .orderBy(f.col('% Volume').desc())\
      .show()

+--------+------------------+-----------------+-------------+--------+
|  Status|      Total/Status|  MedTotal/Status|Vendas/Status|% Volume|
+--------+------------------+-----------------+-------------+--------+
|  Silver|        3014291.36|7631.117367088607|          395|   98.75|
|    Gold|27286.690000000002|9095.563333333334|            3|    0.75|
|Platinum|          12584.68|          6292.34|            2|     0.5|
+--------+------------------+-----------------+-------------+--------+



- Certamente os clientes Platinum são os menos lucrativos. possuem uma fração ínfima do volume de vendas, seu ticket médio e total do valor gerado em vendas é muito menor que os dos status Gold e Silver.

- Gold tem ticket médio acima de Platinum e Silver, entretanto apenas 3 ocorrências. São clientes que possivelmente compram pouco mas produtos com altíssimo valor agregado.

- Silver movimenta a quase totalidade do volume de vendas. Embora seu valor de ticket médio seja abaixo de Gold, sua receita total é também a mais promissora das três categorias.